# 50.007 Machine Learning Project
____

In [81]:
import pandas as pd
import math
import copy
import numpy as np
from collections import Counter, defaultdict


In [82]:
def read_dataset(path, labeled=True):
    with open(path, encoding="utf8") as fp:
        data_read = []
        sentence = []
        for line in fp:
            if line == "\n":
                data_read.append(sentence)
                sentence = []
            else:
                if labeled:
                    tokens = line.strip().split()
                    sentence.append((' '.join(tokens[:-1]), tokens[-1]))
                else:
                    sentence.append((line.strip()))
    return data_read

## 1.1 Calculating MLE
#### **Estimating Emission Parameters**

In [83]:
def get_emissions(data):
    label_counts = defaultdict(int)
    emission_counts = defaultdict(lambda:defaultdict(int))
    for sentence in data:
        for word, label in sentence:
            label_counts[label] += 1
            emission_counts[label][word] += 1
    emissions = defaultdict(float)
    for sentence in data:
        for word,label in sentence:
            emission_prob = emission_counts[label][word]/label_counts[label]
            emissions[(word, label)] = emission_prob
    return emissions

## 1.2 Calculating MLE
#### **Estimating Emission Parameters accounting for unknown words**

In [84]:
def get_e(data, k=1):
    words_list = [x for sentence in data for(x,y) in sentence] 
    #all unique words in the training set
    words = set(words_list)
    label_counts = defaultdict(int)
    emission_counts = defaultdict(lambda:defaultdict(int))
    for sentence in data:
        for word, label in sentence:
            label_counts[label] += 1
            emission_counts[label][word] += 1
    emissions = defaultdict(float)
    for sentence in data:
        for word,label in sentence:
            emission_prob = emission_counts[label][word]/label_counts[label]
            emissions[(word, label)] = emission_prob
    #this function is designed for testing phase so that we can safely handle new words that do not exist in the training set
    def e(x,y): #x:word, y:label
        if x not in words: 
            return k/(label_counts[y] + k)
        else:
            return emissions[(x,y)]
    return e, emissions

## 1.3 Simple Sentiment Analysis
#### **Implementing a simple sentiment analysis system**

In [85]:
def write_file(data, path):
    with open(path, "w", encoding='utf8') as fp:
        for sentence in data:
            for row in sentence:
                fp.write(" ".join(row)+'\n') # rebuild the rows
            fp.write("\n")


In [86]:
def checkforUNK(dev,words):
    dev_2 = copy.deepcopy(dev)
    for sentence in dev_2:
            for i, word in enumerate(sentence):
                if word not in words:
                    sentence[i] = "#UNK#"
    return dev_2

In [87]:
def simple_sentiment(dataset, trainset, output_path):
    labels_list = [y for sentence in trainset for (x,y) in sentence] 
    #all unique words in the training set
    labels = set(labels_list)
    words = set([x for sentence in trainset for (x,y) in sentence])
    dataset = checkforUNK(dataset, words)
    e, emissions = get_e(trainset)
    output=[]
    for sentence in dataset:
        sentence_wlabels=[]
        for word in sentence:
            max_p = 0
            label = None
            for y in labels:
                p = e(word,y)
                if p >max_p:
                    max_p = p
                    label = y
            sentence_wlabels.append((word,label))
        output.append(sentence_wlabels)
    write_file(output, output_path)

In [88]:
ES_dev = read_dataset('./Data/ES/dev.in', labeled=False)
ES_data = read_dataset("./Data/ES/train")
simple_sentiment(ES_dev ,ES_data, "Data/ES/dev.p1.out")
print("ES Evaluation")
!python3 EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p1.out

ES Evaluation

#Entity in gold data: 229
#Entity in prediction: 1466

#Correct Entity : 178
Entity  precision: 0.1214
Entity  recall: 0.7773
Entity  F: 0.2100

#Correct Sentiment : 97
Sentiment  precision: 0.0662
Sentiment  recall: 0.4236
Sentiment  F: 0.1145


In [89]:
RU_dev = read_dataset('./Data/RU/dev.in', labeled=False)
RU_data = read_dataset("./Data/RU/train")
simple_sentiment(RU_dev ,RU_data, "Data/RU/dev.p1.out")
print("RU Evaluation")
!python3 EvalScript/evalResult.py Data/RU/dev.out Data/RU/dev.p1.out

RU Evaluation

#Entity in gold data: 389
#Entity in prediction: 1816

#Correct Entity : 266
Entity  precision: 0.1465
Entity  recall: 0.6838
Entity  F: 0.2413

#Correct Sentiment : 129
Sentiment  precision: 0.0710
Sentiment  recall: 0.3316
Sentiment  F: 0.1170


## 2.1 Estimating Transition Probabilities


In [90]:
def estimate_transition_parameters_test(data):
    transition_counts = defaultdict(lambda: defaultdict(int))
    state_counts = defaultdict(int)
    state_counts['START'] = len(data)
    state_counts['STOP'] = len(data)
    # # Group sentences
    # list_of_sentences = [list(sub) for ele, sub in groupby(sentences, key=bool) if ele]
    # Count state and transition occurrences
    for sentence in data:
        prev_state = "START"
        n = len(sentence)
        for i in range(n):
            state = sentence[i][1]
            state_counts[state] += 1
            if i == n-1:
                transition_counts[prev_state][state] +=1
                transition_counts[state]["STOP"] += 1
            else:
                transition_counts[prev_state][state] += 1 
                prev_state = state
    transitions = defaultdict(lambda: defaultdict(float))
    for prev_state in transition_counts:
        for curr_state in transition_counts[prev_state]:
            transition_prob = transition_counts[prev_state][curr_state]/(state_counts[prev_state])
            transitions[prev_state][curr_state] = transition_prob 
    def q(x,y):
        return transitions[x][y]
    return q, transitions

## 2.2 Viterbi Algorithm

In [91]:
def viterbi_algorithm(sentence, transition_params,e,states):
    n = len(sentence)
    num_states = len(states)
    pi = [{} for i in range(n)]
    backpointers = [{} for i in range(n)]


    for state in states:
        emission_prob = max(e(sentence[0],state),1e-10)
        pi[0][state] = math.log(transition_params['START'].get(state, 1e-10)) + math.log(emission_prob)
        backpointers[0][state] = 'START'

    # Forward pass
    for t in range(1, n):
        for state in states:
            max_prob = float('-inf')
            prev_state = None
            for prev_state in states:
                transition_prob = transition_params[prev_state].get(state, 1e-10)
                emission_prob = max(e(sentence[t],state),1e-10)
                prob = pi[t - 1].get(prev_state, 1e-10) + math.log(transition_prob) + math.log(emission_prob)
                if prob > max_prob:
                    max_prob = prob
                    backpointers[t][state] = prev_state
            pi[t][state] = max_prob

    # Termination step
    max_prob = float('-inf')
    final_state = None
    for state in states:
        transition_prob = transition_params[state].get('STOP', 1e-10)
        prob = pi[n - 1][state] + math.log(transition_prob)
        if prob > max_prob:
            max_prob = prob
            final_state = state

    # Backtracking step
    best_path = [final_state]
    for t in range(n - 1, 0, -1):
        best_path.insert(0, backpointers[t][best_path[0]])

    return best_path

# Run Viterbi algorithm on the development set using viterbi_algorithm_2
def viterbi(dev, transition_params, e,states,words):
    output = []
    dev = checkforUNK(dev,words)
    for sentence in dev:
        best_path = viterbi_algorithm(sentence, transition_params, e, states)
        sentence = list(zip(sentence, best_path)) # new sentence
        output.append(sentence)
    return output

In [92]:
ES_dev = read_dataset('./Data/ES/dev.in', labeled=False)
ES_data = read_dataset("./Data/ES/train")
q, transition_params = estimate_transition_parameters_test(ES_data)
e,emissions = get_e(ES_data, k=1)
states = set([y for sentence in ES_data for (x,y) in sentence ])
words = set([x for sentence in ES_data for (x,y) in sentence ])

output = viterbi(ES_dev, transition_params, e,states,words)
write_file(output, './Data/ES/dev.p2.out')
print("ES Viterbi Evaluation")
!python3 EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p2.out

ES Viterbi Evaluation

#Entity in gold data: 229
#Entity in prediction: 542

#Correct Entity : 134
Entity  precision: 0.2472
Entity  recall: 0.5852
Entity  F: 0.3476

#Correct Sentiment : 97
Sentiment  precision: 0.1790
Sentiment  recall: 0.4236
Sentiment  F: 0.2516


In [93]:
RU_dev = read_dataset('./Data/RU/dev.in', labeled=False)
RU_data = read_dataset("./Data/RU/train")
q, transition_params = estimate_transition_parameters_test(RU_data)
e,emissions = get_e(RU_data, k=1)
states = set([y for sentence in RU_data for (x,y) in sentence ])
words = set([x for sentence in RU_data for (x,y) in sentence ])

output = viterbi(RU_dev, transition_params, e,states,words)
write_file(output, './Data/RU/dev.p2.out')
print("RU Viterbi Evaluation")
!python3 EvalScript/evalResult.py Data/RU/dev.out Data/RU/dev.p2.out

RU Viterbi Evaluation

#Entity in gold data: 389
#Entity in prediction: 489

#Correct Entity : 189
Entity  precision: 0.3865
Entity  recall: 0.4859
Entity  F: 0.4305

#Correct Sentiment : 129
Sentiment  precision: 0.2638
Sentiment  recall: 0.3316
Sentiment  F: 0.2938


## 3.1 Viterbi Algorithm with kth-best sequence

In [94]:
def viterbi_algorithm(sentence, transition_params,e,states, k):
    n = len(sentence)
    num_states = len(states)
    pi = [{} for i in range(n)]
    backpointers = [{} for i in range(n)]


    for state in states:
        emission_prob = max(e(sentence[0],state),1e-10)
        pi[0][state] = math.log(transition_params['START'].get(state, 1e-10)) + math.log(emission_prob)
        backpointers[0][state] = 'START'

    # Forward pass
    for t in range(1, n):
        for state in states:
            max_prob = float('-inf')
            prev_state = None
            prob_list=[]
            for prev_state in states:
                transition_prob = transition_params[prev_state].get(state, 1e-10)
                emission_prob = max(e(sentence[t],state),1e-10)
                prob = pi[t - 1].get(prev_state, 1e-10) + math.log(transition_prob) + math.log(emission_prob)
                prob_list.append((prev_state,prob))
    
            sorted_prob_list = sorted(prob_list, key=lambda x: x[1], reverse=True)
            sorted_states = sorted_prob_list[:k]
            max_prob = sorted_states[-1][1]
            backpointers[t][state] = sorted_states[-1][0]
            # print(backpointers[t][state])
            pi[t][state] = max_prob

    # Termination step
    max_prob = float('-inf')
    final_state = None
    for state in states:
        transition_prob = transition_params[state].get('STOP', 1e-10)
        prob = pi[n - 1][state] + math.log(transition_prob)
        if prob > max_prob:
            max_prob = prob
            final_state = state

    # Backtracking step
    best_path = [final_state]
    for t in range(n - 1, 0, -1):
        best_path.insert(0, backpointers[t][best_path[0]])
    # print(best_path)
    return best_path

# Run Viterbi algorithm on the development set using viterbi_algorithm_2
def modified_viterbi(dev, transition_params, e,states,words,k):
    output = []
    dev = checkforUNK(dev,words)
    for sentence in dev:
        best_path = viterbi_algorithm(sentence, transition_params, e, states,k)
        sentence = list(zip(sentence, best_path)) # new sentence
        output.append(sentence)
    return output

In [95]:
ES_dev = read_dataset('./Data/ES/dev.in', labeled=False)
ES_data = read_dataset("./Data/ES/train")
q, transition_params = estimate_transition_parameters_test(ES_data)
e,emissions = get_e(ES_data, k=1)
states = set([y for sentence in ES_data for (x,y) in sentence ])
words = set([x for sentence in ES_data for (x,y) in sentence ])

best_output = modified_viterbi(ES_dev, transition_params, e,states,words,1)
second_best_output = modified_viterbi(ES_dev, transition_params, e,states,words,2)
eight_best_ouput = modified_viterbi(ES_dev, transition_params, e,states,words,8)

# write_file(best_output, './Data/ES/dev.p3.1st.out')
write_file(second_best_output, './Data/ES/dev.p3.2nd.out')
write_file(eight_best_ouput, './Data/ES/dev.p3.8th.out')

In [96]:
# print("Evaluation for 1st best sequence")
# !python3 EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p3.1st.out
print("\n")
print("Evaluation for 2nd best sequence")
!python3 EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p3.2nd.out
print("\n")
print("Evaluation for 8th best sequence")
!python3 EvalScript/evalResult.py Data/ES/dev.out Data/ES/dev.p3.8th.out




Evaluation for 2nd best sequence

#Entity in gold data: 229
#Entity in prediction: 2252

#Correct Entity : 123
Entity  precision: 0.0546
Entity  recall: 0.5371
Entity  F: 0.0992

#Correct Sentiment : 77
Sentiment  precision: 0.0342
Sentiment  recall: 0.3362
Sentiment  F: 0.0621


Evaluation for 8th best sequence

#Entity in gold data: 229
#Entity in prediction: 4046

#Correct Entity : 191
Entity  precision: 0.0472
Entity  recall: 0.8341
Entity  F: 0.0894

#Correct Sentiment : 98
Sentiment  precision: 0.0242
Sentiment  recall: 0.4279
Sentiment  F: 0.0458


In [97]:
RU_dev = read_dataset('./Data/RU/dev.in', labeled=False)
RU_data = read_dataset("./Data/RU/train")
q, transition_params = estimate_transition_parameters_test(RU_data)
e,emissions = get_e(RU_data, k=1)
states = set([y for sentence in RU_data for (x,y) in sentence ])
words = set([x for sentence in RU_data for (x,y) in sentence ])


best_output = modified_viterbi(RU_dev, transition_params, e,states,words,1)
second_best_output = modified_viterbi(RU_dev, transition_params, e,states,words,2)
eight_best_ouput = modified_viterbi(RU_dev, transition_params, e,states,words,8)

write_file(second_best_output, './Data/RU/dev.p3.2nd.out')
write_file(eight_best_ouput, './Data/RU/dev.p3.8th.out')

In [98]:
print("Evaluation for 2nd best sequence")
!python3 EvalScript/evalResult.py Data/RU/dev.out Data/RU/dev.p3.2nd.out
print("\n")
print("Evaluation for 8th best sequence")
!python3 EvalScript/evalResult.py Data/RU/dev.out Data/RU/dev.p3.8th.out

Evaluation for 2nd best sequence

#Entity in gold data: 389
#Entity in prediction: 3441

#Correct Entity : 163
Entity  precision: 0.0474
Entity  recall: 0.4190
Entity  F: 0.0851

#Correct Sentiment : 108
Sentiment  precision: 0.0314
Sentiment  recall: 0.2776
Sentiment  F: 0.0564


Evaluation for 8th best sequence

#Entity in gold data: 389
#Entity in prediction: 6152

#Correct Entity : 300
Entity  precision: 0.0488
Entity  recall: 0.7712
Entity  F: 0.0917

#Correct Sentiment : 122
Sentiment  precision: 0.0198
Sentiment  recall: 0.3136
Sentiment  F: 0.0373
